In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: FIX DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Fixing all package dependencies and kernel state...')
!pip install huggingface-hub==0.25.2
!pip install sacremoses

# STEP 1: Wipe any incompatible pyarrow
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], 
               capture_output=True, check=False)

# STEP 2: Install pyarrow and essential packages
packages = [
    "pyarrow==15.0.2",
    "transformers==4.38.1", 
    "sentence-transformers==2.2.2",
    "faiss-cpu==1.7.4",
    "rank-bm25==0.2.2",
    "nltk==3.8.1"
]

for package in packages:
    print(f"Installing {package}...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], check=True)

print('✅ All dependencies installed successfully!')
print('\n⚠️  IMPORTANT: Restart kernel now before proceeding to Cell 2')
print('   (Kernel → Restart)')


In [1]:
# ======================== CELL 2: IMPORTS & CONFIGURATION ==========================
import warnings
warnings.filterwarnings("ignore")

import os
import re
import json
import pickle
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np
import torch
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# =============================================================================
# DOMAIN CONFIGURATION
# =============================================================================

@dataclass
class DomainConfig:
    """Configuration for each medical domain"""
    name: str
    dataset_name: str
    index_path: str
    id2doc_path: str
    metadata_path: str

# IMPORTANT: Update these paths to match YOUR Kaggle dataset upload location!
# Example: If you uploaded to "medical-rag-indexes", paths should be:
# "/kaggle/input/medical-rag-indexes/domain_name_faiss.index"

DOMAINS = [
    DomainConfig(
        name="women_health",
        dataset_name="altaidevorg/women-health-mini",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/women_health_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/women_health_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/women_health_metadata.json"
    ),
    DomainConfig(
        name="general_medical",
        dataset_name="lavita/ChatDoctor-HealthCareMagic-100k",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/general_medical_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/general_medical_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/general_medical_metadata.json"
    ),
    DomainConfig(
        name="mental_health",
        dataset_name="Amod/mental_health_counseling_conversations",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/mental_health_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/mental_health_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/mental_health_metadata.json"
    ),
    DomainConfig(
        name="pediatrics",
        dataset_name="lavita/ChatDoctor-iCliniq",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/pediatrics_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/pediatrics_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/pediatrics_metadata.json"
    ),
    DomainConfig(
        name="ophthalmology",
        dataset_name="BaekSeungJu/Ophthalmology-EQA-v3",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/ophthalmology_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/ophthalmology_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/ophthalmology_metadata.json"
    ),
    DomainConfig(
        name="symptoms_triage",
        dataset_name="MedQuad (Kaggle)",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/symptoms_triage_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/symptoms_triage_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/symptoms_triage_metadata.json"
    ),
    DomainConfig(
        name="drug_info",
        dataset_name="ChatDoctor (filtered for drugs)",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/drug_info_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/drug_info_id2doc.pkl",
        metadata_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/drug_info_metadata.json"
    ),
]

# =============================================================================
# RAG CONFIGURATION
# =============================================================================

class RAGConfig:
    """All RAG hyperparameters"""
    # Models
    EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    RERANK_MODEL = "BAAI/bge-reranker-large"
    HYDE_MODEL = "microsoft/BioGPT-Large"
    GENERATOR_MODEL = "microsoft/BioGPT-Large"
    
    # Retrieval
    FAISS_TOP_K = 50
    BM25_TOP_K = 50
    FINAL_TOP_K = 8
    
    # Weights
    FAISS_WEIGHT = 0.6
    BM25_WEIGHT = 0.4
    QUERY_WEIGHT = 0.6
    HYDE_WEIGHT = 0.4
    
    # Generation
    MAX_CONTEXT_LENGTH = 1024
    MAX_ANSWER_LENGTH = 200
    TEMPERATURE = 0.7

config = RAGConfig()

print(f"✅ Configuration loaded")
print(f"📊 Total domains: {len(DOMAINS)}")
print(f"🤖 Models: {config.EMBED_MODEL}, {config.RERANK_MODEL}, {config.GENERATOR_MODEL}")


🔧 Using device: cuda
✅ Configuration loaded
📊 Total domains: 7
🤖 Models: sentence-transformers/all-MiniLM-L6-v2, BAAI/bge-reranker-large, microsoft/BioGPT-Large


In [2]:
# ======================== CELL 3: MULTI-DOMAIN RAG PIPELINE ==========================

class MultiDomainRAGPipeline:
    """
    Production-ready multi-domain medical RAG system
    Features: HyDE, Hybrid Retrieval (FAISS+BM25), Cross-Encoder Reranking, BioGPT Generation
    """
    
    def __init__(self, config: RAGConfig, domains: List[DomainConfig]):
        self.config = config
        self.domains = {}
        self.domain_configs = {d.name: d for d in domains}
        
        print("="*80)
        print("🏥 INITIALIZING MULTI-DOMAIN MEDICAL RAG SYSTEM")
        print("="*80)
        
        # Load models
        self._load_models()
        
        # Load domain indexes
        self._load_domain_indexes(domains)
        
        print(f"\n✅ Pipeline initialized with {len(self.domains)} domains")
        print("="*80)
    
    def _load_models(self):
        """Load all required models"""
        print("\n📦 Loading models...")
        
        # Embedding model
        print(f"  Loading embedder: {self.config.EMBED_MODEL}")
        self.embedder = SentenceTransformer(self.config.EMBED_MODEL, device=device)
        
        # Reranker
        print(f"  Loading reranker: {self.config.RERANK_MODEL}")
        self.reranker = CrossEncoder(self.config.RERANK_MODEL, device=device)
        
        # HyDE model
        print(f"  Loading HyDE model: {self.config.HYDE_MODEL}")
        self.hyde_tokenizer = AutoTokenizer.from_pretrained(self.config.HYDE_MODEL)
        self.hyde_model = AutoModelForCausalLM.from_pretrained(self.config.HYDE_MODEL).to(device)
        print("  ✅ All models loaded")
    
    def _load_domain_indexes(self, domains: List[DomainConfig]):
        """Load pre-built FAISS indexes for all domains"""
        print("\n📂 Loading domain indexes...")
        
        for domain_config in domains:
            try:
                if not os.path.exists(domain_config.index_path):
                    print(f"  ⚠️  Skipping {domain_config.name} (files not found)")
                    continue
                
                print(f"  Loading {domain_config.name}...")
                
                # Load FAISS index
                index = faiss.read_index(domain_config.index_path)
                
                # Load id2doc mapping
                with open(domain_config.id2doc_path, 'rb') as f:
                    id2doc = pickle.load(f)
                
                # Load metadata
                with open(domain_config.metadata_path, 'r') as f:
                    metadata = json.load(f)
                
                # Build BM25 index
                tokenized_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
                bm25 = BM25Okapi(tokenized_corpus)
                
                # Store domain data
                self.domains[domain_config.name] = {
                    'config': domain_config,
                    'faiss_index': index,
                    'bm25_index': bm25,
                    'id2doc': id2doc,
                    'metadata': metadata
                }
                
                print(f"    ✅ Loaded {len(id2doc)} chunks")
                
            except Exception as e:
                print(f"    ❌ Failed: {e}")
                continue
        
        if len(self.domains) == 0:
            raise RuntimeError("No domains loaded! Check your file paths.")
    
    def route_to_domains(self, query: str) -> List[str]:
        """Route query to most relevant domain(s)"""
        query_emb = self.embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True)
        
        scores = []
        for domain_name, domain_data in self.domains.items():
            # Compute domain centroid
            id2doc = domain_data['id2doc']
            sample_docs = id2doc[:min(100, len(id2doc))]
            domain_embs = self.embedder.encode(sample_docs, normalize_embeddings=True, convert_to_numpy=True)
            centroid = np.mean(domain_embs, axis=0, keepdims=True)
            
            # Similarity score
            similarity = np.dot(query_emb, centroid.T)[0][0]
            scores.append((domain_name, float(similarity)))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        
        # Return top 2 domains
        selected = [name for name, score in scores[:2] if score > 0.3]
        
        if not selected and 'general_medical' in self.domains:
            selected = ['general_medical']
        elif not selected:
            selected = [list(self.domains.keys())[0]]
        
        return selected
    
    def generate_hyde(self, query: str) -> str:
        """Generate hypothetical document using HyDE"""
        try:
            prompt = f"Question: {query}\nAnswer:"
            inputs = self.hyde_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
            
            with torch.no_grad():
                outputs = self.hyde_model.generate(
                    **inputs,
                    max_new_tokens=150,
                    temperature=self.config.TEMPERATURE,
                    do_sample=True,
                    pad_token_id=self.hyde_tokenizer.eos_token_id
                )
            
            hyde_answer = self.hyde_tokenizer.decode(outputs[0], skip_special_tokens=True)
            hyde_answer = hyde_answer.split("Answer:")[-1].strip()
            
            return hyde_answer
        except:
            return ""
    
    def hybrid_retrieval(self, query: str, hyde_text: str, domain_names: List[str]) -> List[Dict]:
        """Hybrid retrieval combining FAISS and BM25"""
        # Blend query with HyDE
        blended_query = f"{query} {hyde_text}" if hyde_text else query
        
        all_candidates = []
        
        for domain_name in domain_names:
            if domain_name not in self.domains:
                continue
            
            domain_data = self.domains[domain_name]
            faiss_index = domain_data['faiss_index']
            bm25_index = domain_data['bm25_index']
            id2doc = domain_data['id2doc']
            
            # FAISS search
            query_emb = self.embedder.encode([blended_query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
            D, I = faiss_index.search(query_emb, self.config.FAISS_TOP_K)
            
            faiss_results = {idx: float(score) for idx, score in zip(I[0], D[0]) if idx < len(id2doc)}
            
            # BM25 search
            tokenized_query = word_tokenize(blended_query.lower())
            bm25_scores = bm25_index.get_scores(tokenized_query)
            top_bm25 = np.argsort(bm25_scores)[::-1][:self.config.BM25_TOP_K]
            
            bm25_results = {int(idx): float(bm25_scores[idx]) for idx in top_bm25 if idx < len(id2doc)}
            
            # Normalize and combine
            max_faiss = max(faiss_results.values()) if faiss_results else 1.0
            max_bm25 = max(bm25_results.values()) if bm25_results else 1.0
            
            all_indices = set(faiss_results.keys()) | set(bm25_results.keys())
            
            for idx in all_indices:
                faiss_score = faiss_results.get(idx, 0.0) / max_faiss
                bm25_score = bm25_results.get(idx, 0.0) / max_bm25
                
                combined_score = (
                    self.config.FAISS_WEIGHT * faiss_score +
                    self.config.BM25_WEIGHT * bm25_score
                )
                
                all_candidates.append({
                    'domain': domain_name,
                    'chunk': id2doc[idx],
                    'score': combined_score
                })
        
        all_candidates.sort(key=lambda x: x['score'], reverse=True)
        return all_candidates[:30]
    
    def rerank_results(self, query: str, candidates: List[Dict]) -> List[Dict]:
        """Rerank using cross-encoder"""
        if not candidates:
            return []
        
        pairs = [[query, c['chunk']] for c in candidates]
        rerank_scores = self.reranker.predict(pairs)
        
        for i, cand in enumerate(candidates):
            cand['rerank_score'] = float(rerank_scores[i])
        
        candidates.sort(key=lambda x: x['rerank_score'], reverse=True)
        return candidates[:self.config.FINAL_TOP_K]
    
    def generate_answer(self, query: str, context_chunks: List[Dict]) -> str:
        """Generate final answer using BioGPT"""
        context = "\n\n".join([f"[{i+1}] {c['chunk']}" for i, c in enumerate(context_chunks)])
        
        prompt = f"""Based on the medical information below, answer the question accurately.

Context:
{context}

Question: {query}

Answer:"""
        
        inputs = self.hyde_tokenizer(prompt, return_tensors="pt", max_length=self.config.MAX_CONTEXT_LENGTH, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = self.hyde_model.generate(
                **inputs,
                max_new_tokens=self.config.MAX_ANSWER_LENGTH,
                temperature=self.config.TEMPERATURE,
                do_sample=True,
                pad_token_id=self.hyde_tokenizer.eos_token_id
            )
        
        answer = self.hyde_tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = answer.split("Answer:")[-1].strip()
        
        return answer
    
    def compute_metrics(self, query: str, answer: str, context_chunks: List[Dict]) -> Dict:
        """Compute confidence and quality metrics"""
        # Retrieval score (average rerank)
        retrieval_score = np.mean([c['rerank_score'] for c in context_chunks]) if context_chunks else 0.0
        
        # Faithfulness (answer-context similarity)
        answer_emb = self.embedder.encode([answer], normalize_embeddings=True, convert_to_numpy=True)
        context_text = " ".join([c['chunk'] for c in context_chunks])
        context_emb = self.embedder.encode([context_text], normalize_embeddings=True, convert_to_numpy=True)
        faithfulness = float(np.dot(answer_emb, context_emb.T)[0][0])
        
        # Composite score
        composite = 0.6 * retrieval_score + 0.4 * faithfulness
        
        return {
            'retrieval_score': float(retrieval_score),
            'faithfulness': float(faithfulness),
            'composite': float(composite)
        }
    
    def run_query(self, query: str) -> Dict:
        """Main query processing pipeline"""
        start_time = time.time()
        
        print(f"\n🔍 Query: {query}")
        
        # Route to domains
        selected_domains = self.route_to_domains(query)
        print(f"📍 Domains: {', '.join(selected_domains)}")
        
        # Generate HyDE
        print("🔮 Generating HyDE...")
        hyde_text = self.generate_hyde(query)
        
        # Hybrid retrieval
        print("🔎 Hybrid retrieval...")
        candidates = self.hybrid_retrieval(query, hyde_text, selected_domains)
        print(f"   Retrieved {len(candidates)} candidates")
        
        if not candidates:
            return {
                'query': query,
                'answer': "I apologize, but I couldn't find relevant information to answer your question.",
                'domains': selected_domains,
                'sources': [],
                'metrics': {'composite': 0.0},
                'processing_time': time.time() - start_time
            }
        
        # Rerank
        print("🎯 Reranking...")
        top_chunks = self.rerank_results(query, candidates)
        
        # Generate answer
        print("💬 Generating answer...")
        answer = self.generate_answer(query, top_chunks)
        
        # Compute metrics
        metrics = self.compute_metrics(query, answer, top_chunks)
        
        processing_time = time.time() - start_time
        print(f"✅ Done in {processing_time:.2f}s (confidence: {metrics['composite']:.2f})")
        
        return {
            'query': query,
            'answer': answer,
            'domains': selected_domains,
            'sources': [{'chunk': c['chunk'][:200], 'domain': c['domain'], 'score': c['rerank_score']} 
                       for c in top_chunks],
            'metrics': metrics,
            'processing_time': processing_time
        }

print("✅ MultiDomainRAGPipeline class defined")


✅ MultiDomainRAGPipeline class defined


In [3]:
# ======================== CELL 4: INITIALIZE PIPELINE ==========================

print("\n" + "="*80)
print("🚀 INITIALIZING PIPELINE")
print("="*80 + "\n")

# Initialize the pipeline
pipeline = MultiDomainRAGPipeline(config, DOMAINS)

print("\n" + "="*80)
print("✅ PIPELINE READY!")
print("="*80)



🚀 INITIALIZING PIPELINE

🏥 INITIALIZING MULTI-DOMAIN MEDICAL RAG SYSTEM

📦 Loading models...
  Loading embedder: sentence-transformers/all-MiniLM-L6-v2
  Loading reranker: BAAI/bge-reranker-large
  Loading HyDE model: microsoft/BioGPT-Large
  ✅ All models loaded

📂 Loading domain indexes...
  Loading women_health...
    ✅ Loaded 236304 chunks
  Loading general_medical...
    ✅ Loaded 710919 chunks
  Loading mental_health...
    ✅ Loaded 22565 chunks
  Loading pediatrics...
    ✅ Loaded 19888 chunks
  Loading ophthalmology...
    ✅ Loaded 57979 chunks
  Loading symptoms_triage...
    ✅ Loaded 147907 chunks
  Loading drug_info...
    ✅ Loaded 435395 chunks

✅ Pipeline initialized with 7 domains

✅ PIPELINE READY!


In [4]:
# ======================== CELL 5: INTERACTIVE MODE ==========================

def test_system():
    """Run interactive queries"""
    
    # Predefined test queries
    test_queries = [
        "What are the symptoms of diabetes?",
        "How to treat depression and anxiety?",
        "My 3-year-old has a high fever, what should I do?",
        "What are the side effects of aspirin?",
        "I have blurry vision in my right eye"
    ]
    
    print("\n" + "="*80)
    print("🧪 RUNNING TEST QUERIES")
    print("="*80)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*80}")
        print(f"TEST {i}/{len(test_queries)}")
        print('='*80)
        
        result = pipeline.run_query(query)
        
        print(f"\n💡 Answer:\n{result['answer']}\n")
        print(f"📊 Confidence: {result['metrics']['composite']:.2f}")
        print(f"🎯 Domains: {', '.join(result['domains'])}")
        print(f"⏱️  Time: {result['processing_time']:.2f}s")
        
        if result['sources']:
            print(f"\n📚 Top Sources:")
            for j, source in enumerate(result['sources'][:3], 1):
                print(f"  {j}. [{source['domain']}] Score: {source['score']:.3f}")
                print(f"     {source['chunk']}...")
        
        print("\n" + "-"*80)
    
    print("\n✅ All tests completed!\n")

# Run tests
test_system()

# For custom queries, use:
# result = pipeline.run_query("YOUR QUESTION HERE")
# print(result['answer'])



🧪 RUNNING TEST QUERIES

TEST 1/5

🔍 Query: What are the symptoms of diabetes?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💬 Generating answer...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Done in 4.96s (confidence: 0.53)

💡 Answer:
Choroideremia. < / FREETEXT > < / ABSTRACT > ▃

📊 Confidence: 0.53
🎯 Domains: symptoms_triage
⏱️  Time: 4.96s

📚 Top Sources:
  1. [symptoms_triage] Score: 0.820
     Punctate inner choroidopathy (PIC) is an inflammatory disorder that primarily affects the choroid of the eye and occurs predominantly in young, nearsighted (myopic) women. Signs and symptoms may inclu...
  2. [symptoms_triage] Score: 0.692
     The cause could be the length of the eyeball (longer or shorter), changes in the shape of the cornea, or aging of the lens. Four common refractive errors are       - Myopia, or nearsightedness - clear...
  3. [symptoms_triage] Score: 0.677
     These vision problems are due to an ongoing loss of cells (atrophy) in the specialized light-sensitive tissue that lines the back of the eye (retina) and a nearby network of blood vessels (the choroid...

--------------------------------------------------------------------------------

✅ All test

In [5]:
# ======================== CELL 6: SAVE RESULTS & EXPORT ==========================

# Save sample results to JSON
sample_results = []

test_queries = [
    "What is diabetes?",
    "How to manage anxiety?",
    "Child fever treatment"
]

for query in test_queries:
    result = pipeline.run_query(query)
    sample_results.append({
        'question': query,
        'answer': result['answer'],
        'confidence': result['metrics']['composite'],
        'domains': result['domains']
    })

# Save to file
with open('sample_results.json', 'w') as f:
    json.dump(sample_results, f, indent=2)

print("✅ Sample results saved to sample_results.json")
print("\n📦 TO EXPORT THIS NOTEBOOK:")
print("1. Click the three dots (...) in top right")
print("2. Select 'Download notebook as .py'")
print("3. Send the .py file + all index files to Nikhil")



🔍 Query: What is diabetes?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💬 Generating answer...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Done in 45.65s (confidence: 0.35)
✅ Sample results saved to sample_results.json

📦 TO EXPORT THIS NOTEBOOK:
1. Click the three dots (...) in top right
2. Select 'Download notebook as .py'
3. Send the .py file + all index files to Nikhil
